### Create a calculator tool

In [6]:
# Define a function called calculator that takes a string input `math_problem`
def calculator(math_problem):
    # Use Python's eval() function to evaluate the math expression in the string
    result = eval(math_problem)
    
    # Return the result as a formatted string
    return f"Answer: {result}"

# Call the calculator function w
print(calculator("5 * 12"))

Answer: 60


### Setup our OpenAI Connection

In [7]:
# Import the ChatOpenAI class from the updated langchain_openai package
from langchain_openai import ChatOpenAI

# Import the dotenv function to load environment variables from a .env file
from dotenv import load_dotenv

# Load environment variables from a .env file into the environment
load_dotenv()

# Create an instance of the ChatOpenAI language model
# This instance will use the OpenAI API key from the environment variables
llm = ChatOpenAI()

# Simple test to show our LLM is connected
response = llm.invoke("What is the capital of France?")
print(response.content)


The capital of France is Paris.


### Define tools using Langchain

In [9]:
# Import necessary components for creating and initializing a LangChain agent
from langchain.agents import Tool

# Define a list of tools the agent can use
# In this case, a single tool: a calculator for basic math
tools = [
    Tool(
        name="Calculator",                 # Name of the tool
        func=calculator,                   # The function the tool will call 
        description="Useful for simple math like '2 + 2'"  # Description that helps the agent decide when to use it
    )
]


### Initialize agent

In [10]:
# Import the necessary function and enum from LangChain
from langchain.agents import initialize_agent, AgentType

# Initialize an agent using the specified tools and language model (llm)
# AgentType.ZERO_SHOT_REACT_DESCRIPTION tells the agent to reason and act based on tool descriptions without prior examples
# Setting verbose=True enables detailed logging of the agent's decisions and actions
agent = initialize_agent(
    tools=tools,                        # A list of tools the agent can use
    llm=llm,                            # The language model used by the agent
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent behavior type
    verbose=True                        # Print reasoning steps during execution
)


/var/folders/gb/l1f9l5gs03x861my2hb_s6tw0000gn/T/ipykernel_59989/1390177878.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## Let's run it

In [14]:
result = agent.run("What is 16 * 23")
print(result)



> Entering new AgentExecutor chain...
I can use a calculator to multiply these two numbers together
Action: Calculator
Action Input: 16 * 23
Observation: Answer: 368
Thought:I now know the final answer
Final Answer: 368

> Finished chain.
368


## Let's add a new tool

In [17]:
def get_weather(city):
    """
    Tool: Weather - Returns weather information for major cities
    
    Args:
        city (str): Name of the city to get weather for
        
    Returns:
        int: The current
    """
    # Simulated weather database (in real app, this would call a weather API)
    weather_data = {
        "chicago": 72,
        "london": 65, 
        "tokyo": 87,
        "paris": 59,
        "anchorage": 31
    }
    
    city = city.lower()  # Convert to lowercase for case-insensitive matching
    
    if city in weather_data:
        return f"Weather in {city}: {weather_data[city]}°F"
    else:
        return "City not found"
    
print(get_weather("tokyo"))

Weather in tokyo: 87°F


## Update the Langchain tool list

In [18]:
# Define a list of tools that the agent can use
tools = [
    # A simple calculator tool
    Tool(
        name="Calculator",                          # The name of the tool
        func=calculator,                            # The function that will be executed when this tool is called
        description="Useful for simple math like '2 + 2'"  # A short description of when to use this tool
    ),
    
    # A weather information tool
    Tool(
        name="Weather",                             # The name of the tool
        func=get_weather,                           # The function to retrieve weather details
        description="Useful for getting weather information for major cities"  # Description for when to use this tool
    )
]


## Re-initialize the agent with both tools

In [19]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


## And try the new agent

In [20]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

result = agent.run("What is the temperature in London?")
print(result)



> Entering new AgentExecutor chain...
I need to find out the current temperature in London.
Action: Weather
Action Input: London
Observation: Weather in london: 65°F
Thought:I have obtained the current temperature in London: 65°F.
Final Answer: 65°F

> Finished chain.
65°F


## With a math question...

In [21]:
result = agent.run("What is 7 * 32")
print(result)



> Entering new AgentExecutor chain...
I should use the calculator to solve this multiplication problem.
Action: Calculator
Action Input: 7 * 32
Observation: Answer: 224
Thought:I now know the final answer
Final Answer: 224

> Finished chain.
224


## With two cities...

In [22]:
result = agent.run("What is the temperature in London and Tokyo?")
print(result)



> Entering new AgentExecutor chain...
I should use the Weather tool to get the temperature for each city.
Action: Weather
Action Input: London
Observation: Weather in london: 65°F
Thought:I need to check the temperature for Tokyo as well.
Action: Weather
Action Input: Tokyo
Observation: Weather in tokyo: 87°F
Thought:I now have the temperature for both London and Tokyo.
Final Answer: The temperature in London is 65°F and in Tokyo is 87°F.

> Finished chain.
The temperature in London is 65°F and in Tokyo is 87°F.


## And with a question that needs both tools and multiple calls

In [23]:
result = agent.run("What is the difference in temperature between London and Tokyo")
print(result)



> Entering new AgentExecutor chain...
We need to compare the temperatures in London and Tokyo
Action: Weather
Action Input: London
Observation: Weather in london: 65°F
Thought:We have the weather for London, now we need to check Tokyo
Action: Weather
Action Input: Tokyo
Observation: Weather in tokyo: 87°F
Thought:We have the temperature for both cities, now we can calculate the difference
Action: Calculator
Action Input: 87 - 65
Observation: Answer: 22
Thought:I now know the final answer
Final Answer: The difference in temperature between London and Tokyo is 22°F

> Finished chain.
The difference in temperature between London and Tokyo is 22°F
